# CW1 - Multimodal IMDB Analysis with Keras

## Introduction

In this assignment you will be provided a dataset containing a selection of films together with their posters in JPEG image format and their overviews in text format from the Internet Movie Database.

![Example](https://i.ibb.co/J3trT62/Screenshot-2024-09-22-214753.png)

You will be analysing this dataset by implementing and training two models: a **CNN** and an **LSTM**.

The CNN must classify film posters by the genre. Independently, the LSTM must classify film overviews by the genre. Finally, you will evaluate and critically comment your results in a short report. (Which of the two models was better at classifying films?)

## Structure of the assignment

This assignment is broken up into sections and you need to complete each section successively. The sections are the following:

1. Data Processing

  1.a. Image processing of the posters

  1.b. Natural language processing of the overviews

2. Definition of the models

  2.a. CNN for the posters

  2.b. LSTM for the overviews

3. Training of the models
4. Evaluation of the models

In addition to this coding exercise, you must write a **2-3 pages** report analysing and critically evaluating your model's results. Marks for the report will be awarded for depth of analysis and critical thinking skills. You should consider how well your model performs and WHY it does that—give specific examples and comment on their importance.

In [1]:
# Enter your module imports here, some modules are already provided

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os
import pathlib
import pandas as pd
from sklearn import model_selection
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.metrics import Precision, Recall
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Conv2D, Dropout, MaxPooling2D, Flatten, Dense, Bidirectional,LSTM
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import multilabel_confusion_matrix

In [ ]:
# CodeGrade Tag Init1
from google.colab import drive
drive.mount('/content/drive')

# 1. Data Processing

Warning: running the following cell can take some time.


In [ ]:

# CodeGrade Tag Init2
# Do not edit, remove, or copy this cell

# This code will copy the images from your google drive into the colab file
# storage. Make sure the dataset is unzipped in your drive folder.
! mkdir /tmp/Multimodal_IMDB_dataset
! rsync -ah --info=progress2 /content/drive/MyDrive/Multimodal_IMDB_dataset/Images /tmp/Multimodal_IMDB_dataset/


In [ ]:
# CodeGrade Tag Init3
# Do not edit, remove, or copy this cell

# Load the csv with the sample IDs and genres
csv_loc = "/content/drive/MyDrive/Multimodal_IMDB_dataset/IMDB_overview_genres.csv"
dataset = pd.read_csv(csv_loc)

# Split the dataset into training and testing
data_train, data_test = model_selection.train_test_split(dataset, test_size=0.2)

# Convert the labels to arrays
labels_train = np.array(data_train.drop(['Id', 'Genre', 'Overview'],axis=1)).astype('float32')
labels_test = np.array(data_test.drop(['Id', 'Genre', 'Overview'],axis=1)).astype('float32')

# List of the genre names
genres = np.array(data_train.drop(['Id', 'Genre', 'Overview'],axis=1).columns)

# List of overviews
overviews_train = np.array(data_train['Overview'])
overviews_test = np.array(data_test['Overview'])

# Build the file locations for the images
img_loc = "/tmp/Multimodal_IMDB_dataset/Images"

img_locs_train = np.array([[img_loc + '/' + id + '.jpg' for id in data_train['Id']]])
img_locs_test = [[img_loc + '/' + id + '.jpg' for id in data_test['Id']]]

# This function is provided to read in the image files from the folder
def parse_image(filename, label):
  image = tf.io.read_file(filename)
  image = tf.io.decode_jpeg(image)
  return image, label


### 1.a. Image processing of the posters

In [ ]:
# CodeGrade Tag Init4
# Do not edit, remove, or copy this cell

# Create the initial datasets of film posters
list_posters_train_ds = tf.data.Dataset.from_tensor_slices((img_locs_train[0], labels_train))
list_posters_valid_ds = tf.data.Dataset.from_tensor_slices((img_locs_test[0], labels_test))

* Create a function called ```img_process``` that converts the images to float32 datatype and resizes them to 64x64 pixels

In [ ]:


# CodeGrade Tag Ex1a-i
### Create a function called img_process that converts the images to
### float32 datatype and resizes them to 64x64 pixels

def img_process(image, label):
    #Complete here
    image=tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [64, 64])
    return image, label

* **Using the ``tf.data`` API, load in the training and validation data for the posters. Be mindful of efficient data processing good practice to minimise the time it takes to load the data.**

In [ ]:
# CodeGrade Tag Ex1a-ii
### Use the parse_image and img_process functions to construct the training and
### validation datasets. You should utilise good practice in optimising the
### dataset loading. Use a batch size of 64.

posters_train_ds = list_posters_train_ds.map(parse_image).batch(64).map(img_process,num_parallel_calls=tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
posters_valid_ds = list_posters_valid_ds.map(parse_image).batch(64).map(img_process,num_parallel_calls=tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)

### 1.b. Natural Language processing of the overviews

In [ ]:
# CodeGrade Tag Init5
# Do not edit, remove, or copy this cell

# Create the initial datasets of the film overviews
list_overviews_train_ds = tf.data.Dataset.from_tensor_slices((overviews_train, labels_train))
list_overviews_valid_ds = tf.data.Dataset.from_tensor_slices((overviews_test, labels_test))

* **Using the ``tf.data`` API, load in the training and validation data for the overviews.**

In [ ]:

# CodeGrade Tag Ex1b-i
### Construct the training and validation datasets. Use a batch size of 64.

overviews_train_ds = list_overviews_train_ds.batch(64).prefetch(tf.data.AUTOTUNE)

overviews_valid_ds = list_overviews_valid_ds.batch(64).prefetch(tf.data.AUTOTUNE)

* Build the vocabulary of the model by calling the ``encoder.adapt()`` method on the film overviews train data.

In [ ]:
# CodeGrade Tag Ex1b-ii
### Build the vocabulary of the model by calling the encoder.adapt() method on
### the film overviews train data.

VOCAB_SIZE = 10000

encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(overviews_train_ds.map(lambda text, label: text))

* Print the first 200 words of the vocabulary you obtained.

In [ ]:
# CodeGrade Tag Ex1b-iii
### Print the first 200 words of the vocabulary you obtained.

print(encoder.get_vocabulary()[:200])

# 2. Definition of the models

### 2.a. CNN

**Using the Keras Functional API, create a convolutional neural network with the architecture shown in the model summary below.**

**A few important points to consider:**

* Call the convolutional layers and the first dense layer should have ReLU activation functions. The output layer should have a Sigmoid activation function.
* Pay attention to the output shapes and the number of partmeters for each layer, as these give indications as to the correct settings for the number of filters, kernel size, stride length and padding.
* Use the layer names provided in the summary in your model.
* For the dropout layers, use a dropout rate of 0.2 after the convolutional layers and 0.5 after the dense layers.


```
# Model Summary

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 Input (InputLayer)          [(None, 64, 64, 3)]       0         
                                                                 
 Conv0 (Conv2D)              (None, 32, 32, 16)        448       
                                                                 
 Drop1 (Dropout)             (None, 32, 32, 16)        0         
                                                                 
 Conv1 (Conv2D)              (None, 32, 32, 32)        4640      
                                                                 
 Conv2 (Conv2D)              (None, 32, 32, 32)        9248      
                                                                 
 Drop2 (Dropout)             (None, 32, 32, 32)        0         
                                                                 
 Pool1 (MaxPooling2D)        (None, 16, 16, 32)        0         
                                                                 
 Conv3 (Conv2D)              (None, 16, 16, 64)        18496     
                                                                 
 Conv4 (Conv2D)              (None, 16, 16, 64)        36928     
                                                                 
 Drop3 (Dropout)             (None, 16, 16, 64)        0         
                                                                 
 Pool2 (MaxPooling2D)        (None, 8, 8, 64)          0         
                                                                 
 Conv5 (Conv2D)              (None, 8, 8, 128)         73856     
                                                                 
 Conv6 (Conv2D)              (None, 8, 8, 128)         147584    
                                                                 
 Drop4 (Dropout)             (None, 8, 8, 128)         0         
                                                                 
 Pool3 (MaxPooling2D)        (None, 4, 4, 128)         0         
                                                                 
 Flat (Flatten)              (None, 2048)              0         
                                                                 
 FC1 (Dense)                 (None, 1024)              2098176   
                                                                 
 Drop5 (Dropout)             (None, 1024)              0         
                                                                 
 FC2 (Dense)                 (None, 1024)              1049600   
                                                                 
 Drop6 (Dropout)             (None, 1024)              0         
                                                                 
 Output (Dense)              (None, 25)                25625     
                                                                 
=================================================================
Total params: 3464601 (13.22 MB)
Trainable params: 3464601 (13.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


```



In [ ]:
# CodeGrade Tag Ex2a-i
### Define the model using the Keras Functional API. Use the summary above as a
### guide for the model parameters. You will need to define the filters/units of
### the layers correctly, as well as the kernel size, stride length and padding
### of the convolutional layers.

def build_cnn_model():
  Inp=layers.Input(shape=(64,64,3),name='Input')
  Conv0=layers.Conv2D(filters=16,kernel_size=(3,3)\
                      ,strides=(2,2),padding='same',activation='relu',name='Conv0')(Inp)
  Drop1=layers.Dropout(rate=0.2,name='Drop1')(Conv0)
  Conv1=layers.Conv2D(filters=32,kernel_size=(3,3)\
                      ,strides=(1,1),padding='same',activation='relu',name='Conv1')(Drop1)
  Conv2=layers.Conv2D(filters=32,kernel_size=(3,3)\
                      ,strides=(1,1),padding='same',activation='relu',name='Conv2')(Conv1)
  Drop2=layers.Dropout(rate=0.2,name='Drop2')(Conv2)
  Pool1=layers.MaxPooling2D(pool_size=(2,2)\
                            ,strides=(2,2),name='Pool1')(Drop2)
  Conv3=layers.Conv2D(filters=64,kernel_size=(3,3)\
                      ,strides=(1,1),padding='same',activation='relu',name='Conv3')(Pool1)
  Conv4=layers.Conv2D(filters=64,kernel_size=(3,3)\
                      ,strides=(1,1),padding='same',activation='relu',name='Conv4')(Conv3)
  Drop3=layers.Dropout(rate=0.2,name='Drop3')(Conv4)
  Pool2=layers.MaxPooling2D(pool_size=(2,2)\
                            ,strides=(2,2),name='Pool2')(Drop3)
  Conv5=layers.Conv2D(filters=128,kernel_size=(3,3)\
                      ,strides=(1,1),padding='same',activation='relu',name='Conv5')(Pool2)
  Conv6=layers.Conv2D(filters=128,kernel_size=(3,3)\
                      ,strides=(1,1),padding='same',activation='relu',name='Conv6')(Conv5)
  Drop4=layers.Dropout(rate=0.2,name='Drop4')(Conv6)
  Pool3=layers.MaxPooling2D(pool_size=(2,2)\
                            ,strides=(2,2),name='Pool3')(Drop4)
  Flat=layers.Flatten(name='Flat')(Pool3)
  FC1=layers.Dense(units=1024,activation='relu',name='FC1')(Flat)
  Drop5=layers.Dropout(rate=0.5,name='Drop5')(FC1)
  FC2=layers.Dense(units=1024,activation='relu',name='FC2')(Drop5)
  Drop6=layers.Dropout(rate=0.5,name='Drop6')(FC2)
  Output=layers.Dense(units=25,activation='sigmoid',name='Output')(Drop6)

  model=Model(inputs=Inp,outputs=Output,name='model')

  #Complete here

  return model

cnn_model = build_cnn_model()


* Print the model summary and confirm it has the same architecture as the one provided.

In [ ]:

# CodeGrade Tag Ex2a-ii
### Print the model summary and confirm it has the same architecture as the one
### provided.
cnn_model.summary()
#Complete here

* **Compile the model using the Adam Optimizer with a learning rate of ```1e-4``` and ```binary crossentropy``` loss function. For the metrics, use the ``Precision`` and ``Recall`` functions.**

In [ ]:
# CodeGrade Tag Ex2a-iii
### Compile the model using the Adam Optimizer with a learning rate of 1e-4 and
### binary crossentropy loss function. For the metrics, use the Precision and
### Recall functions.
cnn_model.compile(optimizer=Adam(learning_rate=1e-4),loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),metrics=[Precision(),Recall()])

### 2.b. LSTM model

* Set up the embedding layer by using ```tf.keras.layers.Embedding```. The ```input_dim``` is the length of the vocab, and the ```output_dim``` must be **265**. You should also set ```mask_zero=True```.

In [ ]:
# CodeGrade Tag Ex2b-i
### Set up the embedding layer. The input_dim is the length of the vocab, and
### the output_dim must be 256. You should also set mask_zero=True.

embedder = tf.keras.layers.Embedding(input_dim=VOCAB_SIZE,output_dim=256,mask_zero=True)

* Use ```tf.keras.Sequential``` to build a keras sequential model, with the following layers:



  1.   encoder
  2.   embedder
  3.   biLSTM layer with 256 units, dropout 0.5, recurrent dropout 0.2 (make sure to use the right ```return_sequences``` parametre to be able to stack this layer with the following BiLSTM)
  4.   biLSTM layer with 128 units, dropout 0.5, recurrent dropout 0.2
  5.   dense layer with 128 units and relu activation function
  6.   dropout with rate 0.8
  7.   dense output layer with 25 units and sigmoid activation function



In [ ]:
# CodeGrade Tag Ex2b-ii
### Build a keras sequential model, with the layers provided above.

lstm_model = tf.keras.Sequential([encoder\
                                 ,embedder\
                                 ,Bidirectional(LSTM(units=256,dropout=0.5,recurrent_dropout=0.2,return_sequences=True))\
                                 ,Bidirectional(LSTM(units=128,dropout=0.5,recurrent_dropout=0.2))\
                                 ,Dense(units=128,activation='relu')\
                                 ,Dropout(rate=0.8)\
                                 ,Dense(units=25,activation='sigmoid')])


lstm_model.build((None,))

* Print the model summary and confirm is has the same architecture as the outline provided above.

In [ ]:
# CodeGrade Tag Ex2b-iii
### Print the model summary and confirm is has the same architecture as the
### outline provided above.

lstm_model.summary()

* Compile the model with binary crossentropy loss and the adam optimizer. For the metrics, use the Precision and Recall functions.

In [ ]:
# CodeGrade Tag Ex2b-iv
### Compile the model with binary crossentropy loss, the adam optimizer, with
### the precision and recall metrics

lstm_model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),\
                   optimizer=tf.keras.optimizers.Adam(1e-4),metrics=[Precision(),Recall()])

# 3. Training of the models

* **For each model, create a Checkpoint Callback that saves the weights of the best performing epoch, based on the validation loss.**

In [ ]:
# CodeGrade Tag Ex3a-i
### Create two ModelCheckpoint callbacks to store the bext weights from each
### model, both based on the validation loss.

checkpoint_cnn_filepath = '/content/checkpoint_cnn.weights.h5'
checkpoint_lstm_filepath = '/content/checkpoint_lstm.weights.h5'

from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_cnn_callback = ModelCheckpoint(filepath=checkpoint_cnn_filepath,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True)

checkpoint_lstm_callback = ModelCheckpoint(filepath=checkpoint_lstm_filepath,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True)


* **Create a Learning Rate Scheduler Callback that utilises the provided function to decrease the learning rate during training.**

In [ ]:
# CodeGrade Tag Ex3a-ii
### Using the function provided, create a LearningRateScheduler callback, call
### it "lr_callback"

def scheduler(epoch, lr):
    if epoch < 10:
        return float(lr)
    else:
        return float(lr * tf.math.exp(-0.01))

lr_callback = keras.callbacks.LearningRateScheduler(scheduler)


### 3.a. CNN training

* **Train the CNN model for 40 epochs, using the callbacks you made previously. Store the losses and metrics to use later.**

In [ ]:
# CodeGrade Tag Ex3a-iii
### Train the model for 40 epochs, using the callbacks you have created. Store
### the losses and metrics in a history object.

cnn_history = cnn_model.fit(posters_train_ds\
                            ,epochs=40\
                            ,validation_data=posters_valid_ds\
                            ,callbacks=[lr_callback,checkpoint_cnn_callback])

Epoch 1/40
80/80 ━━━━━━━━━━━━━━━━━━━━ 72s 847ms/step - loss: 0.4225 - precision: 0.2517 - recall: 0.2574 - val_loss: 0.3442 - val_precision: 0.5621 - val_recall: 0.2367 - learning_rate: 1.0000e-04
Epoch 2/40
80/80 ━━━━━━━━━━━━━━━━━━━━ 65s 814ms/step - loss: 0.2581 - precision: 0.4842 - recall: 0.2134 - val_loss: 0.3365 - val_precision: 0.5749 - val_recall: 0.2593 - learning_rate: 1.0000e-04
Epoch 3/40
80/80 ━━━━━━━━━━━━━━━━━━━━ 63s 788ms/step - loss: 0.2464 - precision: 0.5321 - recall: 0.2219 - val_loss: 0.3106 - val_precision: 0.5732 - val_recall: 0.3715 - learning_rate: 1.0000e-04
Epoch 4/40
80/80 ━━━━━━━━━━━━━━━━━━━━ 60s 755ms/step - loss: 0.2409 - precision: 0.5368 - recall: 0.2542 - val_loss: 0.3048 - val_precision: 0.6011 - val_recall: 0.2929 - learning_rate: 1.0000e-04
Epoch 5/40
80/80 ━━━━━━━━━━━━━━━━━━━━ 60s 748ms/step - loss: 0.2363 - precision: 0.5552 - recall: 0.2459 - val_loss: 0.3000 - val_precision: 0.5830 - val_recall: 0.3356 - learning_rate: 1.0000e-04
Epoch 6/40
80/8

* **Train the model for 20 epochs** (this may take several minutes)**, using the callbacks you made previously. Store the losses and metrics to use later.**

In [ ]:
# CodeGrade Tag Ex3b-i
# Train the model for 20 epochs.
lstm_history = lstm_model.fit(overviews_train_ds\
                            ,epochs=20\
                            ,validation_data=overviews_valid_ds\
                            ,callbacks=[lr_callback,checkpoint_lstm_callback])

# 4. Evaluation of the models

### 4.a. CNN Evaluation

* **Create plots using the losses and metrics. In your report, discuss these results and critically evaluate the model performance.**

In [ ]:
# CodeGrade Tag Ex4a-i

fig, ax=plt.subplots(1,3)
fig.set_size_inches(15,5)
fig.suptitle('CNN Model Training | Epochs=40')

ax[0].plot(cnn_history.history['loss'], label='Training')
ax[0].plot(cnn_history.history['val_loss'], label='Validation')
ax[0].legend(loc=0)
ax[0].set_xlabel('Epoch')
ax[0].set_ylabel('Loss')
ax[0].set_title('Model Loss')
ax[0].grid(True)

ax[1].plot(cnn_history.history['precision'], label='Training')
ax[1].plot(cnn_history.history['val_precision'], label='Validation')
ax[1].legend(loc=0)
ax[1].set_xlabel('Epoch')
ax[1].set_ylabel('Precision')
ax[1].set_title('Model Precision')
ax[1].grid(True)

ax[2].plot(cnn_history.history['recall'], label='Training')
ax[2].plot(cnn_history.history['val_recall'], label='Validation')
ax[2].legend(loc=0)
ax[2].set_xlabel('Epoch')
ax[2].set_ylabel('Recall')
ax[2].set_title('Model Recall')
ax[2].grid(True)

plt.show()

* **Load the best weights from your model checkpoint, and create plots demonstrating the classification performance for all three classes. Include these plots in your report, and critically evaluate on the performance of the model across the classes.**

### 4.b. LSTM Evaluation

* **Create plots using the losses and metrics. In your report, discuss these results and critically evaluate the model performance.**

In [ ]:
# CodeGrade Tag Ex4b-i

fig, ax=plt.subplots(1,3)
fig.set_size_inches(15,5)
fig.suptitle('LSTM Model Training | Epochs=20')

ax[0].plot(lstm_history.history['loss'], label='Training')
ax[0].plot(lstm_history.history['val_loss'], label='Validation')
ax[0].legend(loc=0)
ax[0].set_xlabel('Epoch')
ax[0].set_ylabel('Loss')
ax[0].set_title('Model Loss')
ax[0].grid(True)

ax[1].plot(lstm_history.history['precision_1'], label='Training')
ax[1].plot(lstm_history.history['val_precision_1'], label='Validation')
ax[1].legend(loc=0)
ax[1].set_xlabel('Epoch')
ax[1].set_ylabel('Precision')
ax[1].set_title('Model Precision')
ax[1].grid(True)

ax[2].plot(lstm_history.history['recall_1'], label='Training')
ax[2].plot(lstm_history.history['val_recall_1'], label='Validation')
ax[2].legend(loc=0)
ax[2].set_xlabel('Epoch')
ax[2].set_ylabel('Recall')
ax[2].set_title('Model Recall')
ax[2].grid(True)

plt.show()

### 4.c. Produce examples for the report

* First, load the best weights from your checkpoints of both your models.

* Choose a few films from the dataset, plot their posters and print their overviews. Use these example films to demonstrate the classification performance of the CNN model on their posters and of the LSTM model on their overview.

* Be sure to demonstrate the results of the multi-label classification. Compare, for each example film, the top three most probable genres predicted by the CNN and the top three most probable genres predicted by the LSTM with the ground truth genres.

* Include these examples in your report, and critically evaluate on the performance of the model across the classes.

In [ ]:
# CodeGrade Tag Ex4c
cnn_model.load_weights(checkpoint_cnn_filepath)

In [ ]:
lstm_model.load_weights(checkpoint_lstm_filepath)

In [ ]:
from tensorflow.keras.models import save_model, load_model

In [ ]:
cnn_model.save('/content/cnn_model.h5')
lstm_model.save('/content/lstm_model.h5')

cnn_model1 = tf.keras.models.load_model('/content/cnn_model.h5')
lstm_model1 = tf.keras.models.load_model('/content/lstm_model.h5')

cnn_model1.load_weights(checkpoint_cnn_filepath)
lstm_model1.load_weights(checkpoint_lstm_filepath)

#### Select random data to compare

In [ ]:
posters_all=list(posters_valid_ds.unbatch().as_numpy_iterator())
overviews_all=list(overviews_valid_ds.unbatch().as_numpy_iterator())

In [ ]:
random_indices=np.random.choice(len(posters_all),size=10,replace=False)

In [ ]:
posters_ex=[posters_all[i] for i in random_indices]
overviews_ex=[overviews_all[i] for i in random_indices]
images_ex=[data_test['Id'].iloc[i] for i in random_indices]


In [ ]:
for i, (poster, overview,image_id) in enumerate(zip(posters_ex, overviews_ex,images_ex)):
    true_label_poster = poster[1]
    true_label_overview = overview[1]

    prediction_cnn1 = cnn_model.predict(tf.expand_dims(poster[0], axis=0))
    predicted_labels_cnn = (prediction_cnn1 >= 0.5).astype(int)
    prediction_cnn2=[genres[i] for i,value in enumerate(predicted_labels_cnn[0]) if value==1]


    prediction_lstm1 = lstm_model.predict(tf.expand_dims(overview[0], axis=0))
    predicted_labels_lstm = (prediction_lstm1 >= 0.5).astype(int)
    prediction_lstm2=[genres[i] for i,value in enumerate(predicted_labels_lstm[0]) if value==1]


    true=[genres[i] for i,value in enumerate(true_label_poster) if value==1]

    prediction_lstm1_proba=list(zip(genres,prediction_lstm1[0]))
    prediction_lstm1_proba.sort(key=lambda x: x[1], reverse=True)

    prediction_cnn_proba=list(zip(genres,prediction_cnn1[0]))
    prediction_cnn_proba.sort(key=lambda x: x[1], reverse=True)


    img_loc = "/tmp/Multimodal_IMDB_dataset/Images/"
    image = tf.io.read_file(img_loc+image_id+'.jpg')
    image = tf.io.decode_jpeg(image)


    print(f"Example {i+1}",' ImageID:',image_id)
    plt.imshow(image)
    plt.show()
    print("Overview:", overview[0])
    print("True Labels:", true)
    print("Labels (CNN):", prediction_cnn_proba[:3])
    print("Labels (LSTM):", prediction_lstm1_proba[:3])
    print("\n" )

In [ ]:
posters_all_train=list(posters_train_ds.unbatch().as_numpy_iterator())
Total_Train=list(np.array([i[1] for i in posters_all_train]).sum(axis=0))
Total_Test=list(np.array([i[1] for i in posters_all]).sum(axis=0))

In [ ]:
#Distribution of test samples
plt.bar(genres,Total_Test)
plt.title('Distribution of Samples by Classes | Total Test')
plt.xlabel('Genres')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.grid(True)
plt.show()

In [ ]:
#Distribution of train samples
plt.bar(genres,Total_Train)
plt.title('Distribution of Samples by Classes | Total Train')
plt.xlabel('Genres')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.grid(True)
plt.show()

In [ ]:
predictions=cnn_model.predict(posters_valid_ds)

In [ ]:
predictions[predictions>=0.5]=1
predictions[predictions<0.5]=0

In [ ]:
true_labels_test=np.array([i[1] for i in posters_all])

In [ ]:
confusion_m1=multilabel_confusion_matrix(true_labels_test,predictions)

In [ ]:
#CNN Predictions
fig,ax=plt.subplots(5,5,sharex=True,sharey=True)
fig.set_size_inches(15,15)
ax=ax.flatten()
for n,i in enumerate(confusion_m1):
  cm=pd.DataFrame(i,index=['True 0','True 1'],columns=['Pred 0','Pred 1'])
  sns.heatmap(cm,annot=True,fmt='d',cmap='Blues',ax=ax[n])
  #plt.title('Confusion Matrix')
  ax[n].set_title(genres[n])
  #print(i)

In [ ]:
overviews_all_train=list(overviews_train_ds.unbatch().as_numpy_iterator())
Total_Train=list(np.array([i[1] for i in overviews_all_train]).sum(axis=0))
Total_Test=list(np.array([i[1] for i in overviews_all]).sum(axis=0))
predictions=lstm_model.predict(overviews_valid_ds)
predictions[predictions>=0.5]=1
predictions[predictions<0.5]=0
true_labels_test=np.array([i[1] for i in overviews_all])
confusion_m1=multilabel_confusion_matrix(true_labels_test,predictions)

In [ ]:
#LSTM Predictions
fig,ax=plt.subplots(5,5,sharex=True,sharey=True)
fig.set_size_inches(15,15)
ax=ax.flatten()
for n,i in enumerate(confusion_m1):
  cm=pd.DataFrame(i,index=['True 0','True 1'],columns=['Pred 0','Pred 1'])
  sns.heatmap(cm,annot=True,fmt='d',cmap='Blues',ax=ax[n])
  #plt.title('Confusion Matrix')
  ax[n].set_title(genres[n])
  #print(i)